<a id="top"></a>
# Astroquery: Exploring Metadata from the Nancy Grace Roman Space Telescope
***
## Learning Goals

By the end of this tutorial, you will:

- Understand how to use the `astroquery.mast` module to access metadata from the Roman Space Telescope.
- Run metadata queries based on coordinates, an object name, or non-positional criteria.
- Use optional search parameters to further refine query results.

## Table of Contents
* [Introduction](#Introduction)
* [Querying MAST for Roman Metadata](#Querying-MAST-for-Roman-Metadata)
    * [Setup](#Setup)
    * [Optional Search Parameters](#Optional-Search-Parameters)
    * [Query by Criteria](#Query-by-Criteria)
    * [Roman Product Downloads](#Downloading-and-Opening-Roman-Data-Products)
* [Additional Resources](#Additional-Resources)
* [Exercise Solutions](#Exercise-Solutions)

## Introduction

Welcome! This tutorial focuses on using the `astroquery.mast` module to search for metadata from the [Nancy Grace Roman Space Telescope](https://roman.gsfc.nasa.gov/). Scheduled for launch by May 2027, Roman is an advanced survey telescope designed for observations in the infrared light spectrum.

The [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/) hosts publicly accessible data products from space telescopes like Roman. `astroquery.mast` provides access to a broad set of Roman metadata, including header keywords, proposal information, and observational parameters. The available metadata can also be found using the [MAST Roman Search](https://mast.stsci.edu/search/ui/#/roman) interface.

<div class="alert alert-info">
Please note that pre-launch, <b>`astroquery.mast.MastMissions` and the MAST Roman Search API require authorization to search and download Roman data products.</b> Before we get started, please ensure that:
    
- ***you are authorized to search and download Roman engineering data from MAST.*** If you are not authorized but you think you should be, email the helpdesk at archive@stsci.edu
- ***you have a [MAST token](https://auth.mast.stsci.edu/token) set to the environment variable `MAST_API_TOKEN`***
</div>

<div class="alert alert-warning" style="color:black; background-color:#ffc5c5; border-color:red;">
<b>Note</b> that at this time, Roman data are not accessible from the cloud with `astroquery.mast`. Downloads will come from MAST servers and may be large. Download with caution.
</div>

<div class="alert alert-info" style="color:black; background-color:orange; border-color:black">
<b>TODO</b>
    
- flesh out section on product downloads
- add exercises with solutions for Roman searches
</div>

## Imports

This notebook uses the following packages:
- `os` to get the `MAST_API_TOKEN` from the environment variables
- `astroquery.mast` to query the MAST Archive

In [ ]:
import os
from astroquery.mast import MastMissions

***

## Querying MAST for Roman Metadata

### Setup

In order to make queries on Roman metadata, we will have to perform some setup. First, we will instantiate an object of the `MastMissions` class, assign its `mission` to be `'roman'`, and login with our MAST token. Its `service` is set to the default of `'search'`.

In [ ]:
# Create MastMissions object and assign mission to 'roman'
missions = MastMissions(mission='roman')

# Login to search and retrieve Roman data
missions.login(token=os.getenv("MAST_API_TOKEN"))
               
print(f'Mission: {missions.mission}')
print(f'Service: {missions.service}')

When writing queries, keyword arguments can be used to specify output characteristics (see the following section) and filter on values like instrument, exposure type, and proposal ID. The available column names for a mission are returned by the `get_column_list` function. Below, we will print out the name, data type, and description for the first 10 columns in Roman metadata.

In [ ]:
# Get available columns for Roman mission
columns = missions.get_column_list()
columns[:10]

### Optional Search Parameters

Before we dive in to the actual queries, it's important to know how we can refine our results with optional keyword arguments. The following parameters are available:

- `limit`: The maximum number of results to return. Default is 5000.
- `offset`: Skip the first ***n*** results. Useful for paging through results.
- `select_cols`: A list of columns to be returned in the response.

395 JWST datasets fall within our cone search. In other words, their target coordinates are within 10 arcminutes of the coordinates that we defined.

### Query by Criteria

In some cases, we may want to run queries with non-positional parameters. To accomplish this, we use the `query_criteria` function.

For any of our query functions, we can filter our results by the value of columns in the dataset.

Let's say that we only want observations from the test Galactic Bulge Time Domain Survey (GBTDS), which was executed during Mission Readiness Test 8. The test GBTDS run corresponds to program 163.

In [ ]:
# Query with column criteria
results = missions.query_criteria(
    program=163,
    select_cols=[
        'fileSetName', 'detector', 'productLevel', 
        'product_type', 'exposure_type', 'instrument_name', 
        'optical_element', 'exposure_time', 'program',
    ],
)

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:5]

To exclude and filter out a certain value from the results, we can prepend the value with `!`.

Let's run the same query as above, but this time, we will filter out datasets coming from the WFI02 detector.

In [ ]:
# Query with exclude criteria
results = missions.query_criteria(
    program=163,
    detector='!WFI02',
    select_cols=[
        'fileSetName', 'detector', 'productLevel', 
        'product_type', 'exposure_type', 'instrument_name', 
        'optical_element', 'exposure_time', 'program',
    ],
)

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:5]

For columns with numeric or date values, we can filter using comparison values:

- `<`: Return values less than or before the given number/date
- `>`: Return values greater than or after the given number/date
- `<=`: Return values less than or equal to the given number/date
- `>=`: Return values greater than or equal to the given number/date

As an example, let's write a query to return all Program 163 datasets with an exposure start time after 12 PM UTC on September 12, 2025.

In [ ]:
# Query using comparison operator
results = missions.query_criteria(
    program=163,
    exposure_start_time='> 2025-09-12 12:00:00',
    select_cols=[
        'fileSetName', 'detector', 'productLevel', 
        'product_type', 'optical_element', 'exposure_start_time',
    ],
)

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:5]

For numeric or date data types, we can also filter with ranges. This requires the following syntax: `'#..#'`.

Let's write a query that uses range syntax to return datasets that belong to a program number between 161 and 164. We will also select for exposure durations that are greater than or equal to 100 seconds.

In [ ]:
# Query using range operator
results = missions.query_criteria(program='161..164', # Program number between 1150 and 1155
                                  exposure_time='>100',  # Exposure duration is greater than or equal to 100 seconds
                                  select_cols=['fileSetName', 'program', 'exposure_time'])

# Display results
print(f'Total number of results: {len(results)}')
results

### Downloading and Opening Roman Data Products

explainy text

In [ ]:
# Query with column criteria
results = missions.query_criteria(
    program=163,
    select_cols=[
        'fileSetName', 'detector', 'productLevel', 
        'product_type', 'exposure_type', 'instrument_name', 
        'optical_element', 'exposure_time', 'program',
    ],
    limit=5,
)

results

explainy text

In [ ]:
products = missions.get_product_list(results)
products

explainy text

In [ ]:
files = missions.download_products(products)
files

## Additional Resources

- [MAST Roman Search Form](https://mast.stsci.edu/search/ui/#/Roman)
- [MAST Roman Search API](https://mast.stsci.edu/search/docs/?urls.primaryName=roman_api)
- [`astroquery.mast` Documentation for Mission-Specific Searches](https://astroquery.readthedocs.io/en/latest/mast/mast_missions.html#mission-specific-search-queries)

## Citations

If you use `astroquery` for published research, please cite the
authors. Follow these links for more information about citing `astroquery`:

* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)


## About this Notebook

**Author(s):** Zach Claytor and Sedona Price, adapted from [JWST MAST Metadata Search](https://github.com/zclaytor/mast_notebooks/blob/main/notebooks/JWST/MAST_metadata_search/MAST_metadata_search.ipynb) by Sam Bianco <br>
**Keyword(s):** Tutorial, Roman, Astroquery, MastMissions <br>
**First published:** Feb 2026 <br>
**Last updated:** Feb 2026 <br>

***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/hst_notebooks/blob/main/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 